In [1]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
import csv
import monai
from monai.transforms import \
    Compose, LoadImaged, AddChanneld, Orientationd, \
    Spacingd, \
    ToTensord,  \
    DataStatsd, \
    ToDeviced
from monai.data import list_data_collate
import torch
import pytorch_lightning as pl
from torchsummary import summary
monai.config.print_config()
import sys
# sys.path.append(r'/data16/private/zc348/project/DL_HMC_attention/mCT/util/python')
sys.path.append(r'/data16/private/zc348/project/DL_HMC_attention/util/python')
import vicra_toolbox
import nibabel; nibabel.imageglobals.logger.setLevel(40)
# New transforms 

sys.path.append(r'../')

torch.set_num_threads(4)
from dlhmc.transforms import (
    CreateImageStack,
    ComputeRelativeMotion,
    RandSamplePET,
    ComputeRelativeMotiond,
    CreateImageStackd,
    RandSamplePETd,
)

from dlhmc.utils.data import (
    concatenate_vicra,
    split_dataset
)



2024-04-08 05:30:45,697 - Created a temporary directory at /tmp/tmplx4a3rn6
2024-04-08 05:30:45,701 - Writing /tmp/tmplx4a3rn6/_remote_module_non_scriptable.py
MONAI version: 1.0.1
Numpy version: 1.23.4
Pytorch version: 1.13.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 8271a193229fe4437026185e218d5b06f7c8ce69
MONAI __file__: /home1/zc348/anaconda3/envs/dl-hmc_2301c/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 4.0.2
scikit-image version: NOT INSTALLED or UNKNOWN VERSION.
Pillow version: 9.2.0
Tensorboard version: 2.11.0
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.14.1
tqdm version: 4.64.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.0
pandas version: 1.4.3
einops version: 0.6.0
transformers version: NOT INSTALLED or UNKNOWN VERSION.
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd versi

In [2]:
tr_set_all=['AH017', 'AL819', 'AL831', 'AO822', 'AT653', 'AT687', 'BB302',
       'BD233', 'BJ464', 'BK384', 'MM484', 'BR043', 'BW083', 'CA557',
       'CB411', 'CD348', 'CF952', 'CH144', 'CK217', 'CM087', 'CM712',
       'DB031', 'DB371', 'DB781', 'DF607', 'DG649', 'DH285', 'DM113',
       'DM524', 'DS880', 'EC569', 'EC599', 'EG252', 'EK470', 'ER338',
       'ET172', 'EW491', 'FF840', 'GB696', 'GL357', 'HW920', 'JB080',
       'JB472', 'JB737', 'JD861', 'JF625', 'JH281', 'JR857', 'JS053',
       'JS270', 'JS315', 'KA848', 'KC328', 'KN938', 'KR673', 'KR673',
       'KS017', 'KS017', 'KS185', 'LC157', 'LJ271', 'LL150', 'LL783',
       'LS439', 'MC549', 'MD602', 'MF885', 'MJ068', 'MJ291', 'MK283',
       'MM520', 'MR627', 'NC298', 'NF861', 'NJ610', 'NP335', 'PC682',
       'RC963', 'RH479', 'RK887', 'RM802', 'RS467', 'RT805', 'SA836',
       'SE511', 'SH996', 'SK238', 'SL436', 'SL855', 'SM529', 'SR015',
       'TB254', 'TB937', 'TF645', 'TH497', 'TK231', 'UB142', 'WB408',
       'WL971', 'WM493']

In [5]:
[i in tr_set_all for i in test_set]

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [2]:
test_set=['PF605', 'JB538', 'AF120', 'JG369', 'SY636',
'EC950', 'NM937', 'AS469', 'CH568', 'JO308',
'CJ509', 'SY869', 'BB688', 'HR322', 'DS636',
 'JR684', 'JM100', 'SM968', 'TM628', 'MC181']

In [2]:
tr_set_all=['AH017', 'AL819', 'AL831', 'AO822', 'AT653', 'AT687', 'BB302',
       'BD233', 'BJ464', 'BK384', 'MM484', 'BR043', 'BW083', 'CA557',
       'CB411', 'CD348', 'CF952', 'CH144', 'CK217', 'CM087', 'CM712',
       'DB031', 'DB371', 'DB781', 'DF607', 'DG649', 'DH285', 'DM113',
       'DM524', 'DS880', 'EC569', 'EC599', 'EG252', 'EK470', 'ER338',
       'ET172', 'EW491', 'FF840', 'GB696', 'GL357', 'HW920', 'JB080',
       'JB472', 'JB737', 'JD861', 'JF625', 'JH281', 'JR857', 'JS053',
       'JS270', 'JS315', 'KA848', 'KC328', 'KN938', 'KR673', 'KR673',
       'KS017', 'KS017', 'KS185', 'LC157', 'LJ271', 'LL150', 'LL783',
       'LS439', 'MC549', 'MD602', 'MF885', 'MJ068', 'MJ291', 'MK283',
       'MM520', 'MR627', 'NC298', 'NF861', 'NJ610', 'NP335', 'PC682',
       'RC963', 'RH479', 'RK887', 'RM802', 'RS467', 'RT805', 'SA836',
       'SE511', 'SH996', 'SK238', 'SL436', 'SL855', 'SM529', 'SR015',
       'TB254', 'TB937', 'TF645', 'TH497', 'TK231', 'UB142', 'WB408',
       'WL971', 'WM493']

test_set=['PF605', 'JB538', 'AF120', 'JG369', 'SY636',
'EC950', 'NM937', 'AS469', 'CH568', 'JO308',
'CJ509', 'SY869', 'BB688', 'HR322', 'DS636',
 'JR684', 'JM100', 'SM968', 'TM628', 'MC181']
# test_set=['AO101', 'PF605', 'JO308', 'AS469', 'JR684']
tr_set = tr_set_all[0:100]
from dataset_summary_toolbox import compute_delta_T 
from data_prep_toolbox import delta_T_magnitude, Relative_motion_A_to_B_12, build_legal_dataset, deal_dataframe, clean_df
from sampling_toolbox import data_split_sample, add_T_deltaT

# Get the summaries

summaries, delta_T_all=compute_delta_T(['FDG'], tr_set, data_type = 'real')

def calculate_beta(df):
    nums = df['T'].to_numpy()
    trans, rot = 0, 0
    for array in range(len(nums)):
        for i in range(3):
            trans += abs(nums[array][i])
        for j in range(3):
            rot += abs(nums[array][3 + j])
#     print("trans:", trans, "rot:", rot)
    beta = trans/(trans + rot)
#     print('beta:', beta)
    return beta

num_subsample = 360 #1800 # Number of samples per subject (to be split between training, validation and testing sets)

train_list=list()
val_list=list()
test_list=list()

# Sample training, validation and testing sets and save them in lists
for summary in summaries: 
    summary['VICRA']=concatenate_vicra(summary)

    print('Dataset contains total {:,d} entries'.format(len(summary)))
    df_train, df_val, df_test = split_dataset(summary, num_subsample=num_subsample, test_size_percent=0.1, validation_size_percent=0.1)
    
    # train_beta_array = df_train.groupby('PatientID').apply(calculate_beta)
    # train_beta_array = pd.DataFrame({'PatientID':train_beta_array.index, 'beta':train_beta_array.values})
    # df_train = pd.merge(df_train, train_beta_array, on="PatientID")

    # val_beta_array = df_val.groupby('PatientID').apply(calculate_beta)
    # val_beta_array = pd.DataFrame({'PatientID':val_beta_array.index, 'beta':val_beta_array.values})
    # df_val = pd.merge(df_val, val_beta_array, on="PatientID")

    # test_beta_array = df_test.groupby('PatientID').apply(calculate_beta)
    # test_beta_array = pd.DataFrame({'PatientID':test_beta_array.index, 'beta':test_beta_array.values})
    # df_test = pd.merge(df_test, test_beta_array, on="PatientID")
    print('Dataset split into 3 subsets:')
    print(' Train set size: {:,d}'.format(len(df_train)))
    print(' Val set size: {:,d}'.format(len(df_val)))
    print(' Test set size: {:,d}'.format(len(df_test)))
    
    train_dict={
        'DataFrame':df_train
    }

    val_dict={
        'DataFrame':df_val
    }
    
    test_dict={
        'DataFrame':df_test
    }
    
    train_list.append(train_dict)
    val_list.append(val_dict)
    test_list.append(test_dict)


    
# SIZE = (64,64,64)
SIZE = (32,32,32)

image_key = "ThreeD_Cloud"
meta_keys=['ScanStart','VICRA']

# Need to format as a dictionary for input
# sample_dict = {
#     'DataFrame': df_train,
    # 'PatientID': df_sample['PatientID'].unique()
# }

# Here, we only have one subject of data, so a small example, but more subjects can be added
# to this list.
# sample_data_dict = [sample_dict]

pet_transforms = monai.transforms.Compose([
    CreateImageStackd(keys="DataFrame", image_key=image_key, spatial_size=SIZE),
    RandSamplePETd(keys=image_key, meta_data_key="DataFrame", meta_keys=meta_keys, num_samples=8), 
    ComputeRelativeMotiond(keys=['VICRA_ref','VICRA_mov'], output_key='VICRA_rel'),
    monai.transforms.DeleteItemsd(keys=["DataFrame", "ScanStart", "VICRA_ref", "VICRA_mov", image_key]),
    monai.transforms.ToTensord(
        keys=["ScanStart_ref", "ScanStart_mov", "VICRA_rel"]
    )
])



train_ds = monai.data.PersistentDataset(
    data=train_list, 
    transform=pet_transforms, 
    cache_dir="/data16/private/zc348/project/dev/100hrrt/cache/"
)

train_loader = torch.utils.data.DataLoader(
    train_ds, 
    batch_size=12, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=monai.data.list_data_collate
)

val_ds = monai.data.PersistentDataset(
    data=val_list, 
    transform=pet_transforms, 
    cache_dir="/data16/private/zc348/project/dev/100hrrt/cache/"
)

val_loader = torch.utils.data.DataLoader(
    val_ds, 
    batch_size=12, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=monai.data.list_data_collate
)







Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36
Dataset contains total 1,800 entries
Dataset split into 3 subsets:
 Train set size: 288
 Val set size: 36
 Test set size: 36


In [11]:
import torch.nn as nn
import torch.nn.functional as F


In [4]:

class UNet(nn.Module):

    def __init__(self, in_channel, out_channel):
        super(UNet, self).__init__()
        #Encode
        self.conv_encode1 = self.contracting_block(in_channels=in_channel, out_channels=32)
        self.conv_maxpool1 = torch.nn.MaxPool3d(kernel_size=2)
        self.conv_encode2 = self.contracting_block(32, 64)
        self.conv_maxpool2 = torch.nn.MaxPool3d(kernel_size=2)
        self.conv_encode3 = self.contracting_block(64, 128)
        self.conv_maxpool3 = torch.nn.MaxPool3d(kernel_size=2)


    def contracting_block(self, in_channels, out_channels, kernel_size=3):
        """
        This function creates one contracting block
        """
        block = torch.nn.Sequential(
            torch.nn.Conv3d(kernel_size=kernel_size, in_channels=in_channels, out_channels=out_channels, padding=1),
            torch.nn.BatchNorm3d(out_channels),
            torch.nn.ReLU(),
            torch.nn.Conv3d(kernel_size=kernel_size, in_channels=out_channels, out_channels=out_channels, padding=1),
            torch.nn.BatchNorm3d(out_channels),
            torch.nn.ReLU(),
        )
        return block
    
    def forward(self, x):
        # Encode
        encode_block1 = self.conv_encode1(x)
        encode_pool1 = self.conv_maxpool1(encode_block1)
        encode_block2 = self.conv_encode2(encode_pool1)
        encode_pool2 = self.conv_maxpool2(encode_block2)
        
        encode_block3 = self.conv_encode3(encode_pool2)
        encode_pool3 = self.conv_maxpool3(encode_block3)


        return encode_pool3

In [5]:
class Cross_attention(nn.Module):
    def  __init__(self, all_channel=128, all_dim=4*4*4):	

        super(Cross_attention, self).__init__()
        self.linear_e = nn.Linear(all_channel, all_channel,bias = False)
        self.channel = all_channel
        self.dim = all_dim
        self.gate = nn.Conv3d(all_channel, 1, kernel_size  = 1, bias = False)
        self.gate_s = nn.Sigmoid()
        self.conv1 = nn.Conv3d(all_channel*1, all_channel, kernel_size=3, padding=1, bias = False)
        self.conv2 = nn.Conv3d(all_channel*1, all_channel, kernel_size=3, padding=1, bias = False)
        self.bn1 = nn.BatchNorm3d(all_channel)
        self.bn2 = nn.BatchNorm3d(all_channel)
        self.prelu = nn.ReLU(inplace=True)
        self.conv3d_7 = nn.Conv3d(in_channels=all_channel *2 , out_channels=all_channel, kernel_size=1, stride=(1, 1, 1), padding=0)
        self.pathC_bn1 = nn.BatchNorm3d(all_channel*2)
        self.conv3d_8 = nn.Conv3d(in_channels=all_channel, out_channels=all_channel//2, kernel_size=3, stride=(1, 1, 1), padding=1)
        self.conv3d_9 = nn.Conv3d(in_channels=all_channel//2, out_channels=all_channel//8, kernel_size=3, stride=(1, 1, 1), padding=1)
        self.pathC_bn2 = nn.BatchNorm3d(all_channel//8)

        self.conva = nn.Conv3d(all_channel, all_channel, kernel_size=1, padding=0, bias = False)
        self.convb = nn.Conv3d(all_channel, all_channel, kernel_size=1, padding=0, bias = False)
    
    
		
    def forward(self, exemplar, query): 
        
	 
        fea_size = query.size()[2:]	 
        
        #### correlation matrix computation
        exemplar_v =  self.convb(exemplar) 
        query_v =  self.convb(query) 
        exemplar_q =  self.conva(exemplar) 
        query_k =  self.conva(query) 
        exemplar_flat = exemplar_q.view(-1, self.channel, self.dim) #N,C,H*W
        query_flat = query_k.view(-1, self.channel, self.dim)
        exemplar_t = torch.transpose(exemplar_flat,1,2).contiguous()  #batch size x dim x num
        A = torch.bmm(exemplar_t, query_flat)
        A = F.softmax(A, dim = 1) 
        B = F.softmax(torch.transpose(A,1,2),dim=1)
        query_att = torch.bmm(exemplar_v.view(-1, self.channel, self.dim) , A).contiguous() 
        exemplar_att = torch.bmm(query_v.view(-1, self.channel, self.dim), B).contiguous()
        #####self-gate mechanism
        input1_att = exemplar_att.view(-1, self.channel, fea_size[0], fea_size[1], fea_size[2])  
        input2_att = query_att.view(-1, self.channel, fea_size[0], fea_size[1], fea_size[2])
        input1_mask = self.gate(input1_att)
        input2_mask = self.gate(input2_att)
        input1_mask = self.gate_s(input1_mask)
        input2_mask = self.gate_s(input2_mask)
        input1_att = input1_att * input1_mask
        input2_att = input2_att * input2_mask
        input1_att = input1_att +  exemplar_v
        input2_att = input2_att + query_v
        ######Deep norm and fusion
        input1_att  = self.conv1(input1_att )
        input2_att  = self.conv2(input2_att ) 
        input1_att  = self.bn1(input1_att )
        input2_att  = self.bn2(input2_att )
        input1_att  = self.prelu(input1_att )
        input2_att  = self.prelu(input2_att )
        conv_input = torch.cat((exemplar, query), 1)

        # conv_input = torch.cat((input1_att, input2_att), 1)
        x = self.pathC_bn1(conv_input)
        x = self.conv3d_7(x)
        x = self.prelu(x)
        x = self.conv3d_8(x)
        x = self.prelu(x)
        x = self.conv3d_9(x)
        x = self.pathC_bn2(x)

        return x.view(x.size()[0],-1)

In [6]:
from dlhmc.utils import Relative_motion_A_to_B_12,RotTransMatrix_6Params

def matrix_transformation(gt_reg):
    i_6_list = list()
    for i_12 in gt_reg:
        i_6 = np.reshape(i_12,12)
        i_6 = torch.from_numpy(RotTransMatrix_6Params(i_6,1))
        i_6_list.append(i_6)
        
    gt_reg_6 = torch.stack(i_6_list).float()
    # return gt_reg_6
    return gt_reg_6.to(gt_reg.get_device())

In [9]:
class PETRegNet_dataloader(pl.LightningModule):
    
    def __init__(self, dropout=0.3,img_size=32):
        super().__init__()
        
             
        self.dropout = dropout
        
        self.feature_extractor = UNet(1,1)

        self.coattention = Cross_attention()


        self.regression_layers = torch.nn.Sequential(

            torch.nn.Linear(1024, 128),
            torch.nn.Linear(128, 16),
            torch.nn.Linear(16, 6),
        )
        self.loss_function = torch.nn.MSELoss()
        
    def forward(self, x1, x2):

        y1 = self.feature_extractor(x1)
        y2 = self.feature_extractor(x2)
        y = self.coattention(y1,y2)
        y = self.regression_layers(y)
        return y
    
    
    
    def prepare_data(self):
        # set deterministic training for reproducibility
        monai.utils.misc.set_determinism(seed=42)
    
    def training_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        ref_time = batch['ScanStart_ref']
        mov_time = batch['ScanStart_mov']
        # x_t = torch.stack([ref_time, mov_time], dim=1)
        gt_reg = batch["VICRA_rel"].float()
        y = self.forward(x1,x2)
       
        target_six = matrix_transformation(gt_reg)
        loss = self.loss_function(y, target_six)   

        return {"loss": loss}

        
    def training_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Train/Loss', avg_loss, self.current_epoch)
    
    def validation_step(self, batch, batch_idx):
        x1 = batch["ThreeD_Cloud_ref"]
        x2 = batch["ThreeD_Cloud_mov"]
        ref_time = batch['ScanStart_ref']
        mov_time = batch['ScanStart_mov']
        x_t = torch.stack([ref_time, mov_time], dim=1)
        gt_reg = batch["VICRA_rel"].float()
        y = self.forward(x1,x2)
        target_six = matrix_transformation(gt_reg)
        loss = self.loss_function(y, target_six)   

        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        # Calculate the average loss
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        # Logging at the end of every epoch
        self.logger.experiment.add_scalar('Val/Loss', avg_loss, self.current_epoch)

        # Log the value for model checkpoint saving
        self.log('val_loss', avg_loss.item()) #added .item(), otherwise validation wouldn't work 

    def configure_optimizers(self):
        total_params = list(self.feature_extractor.parameters()) + list(self.regression_layers.parameters()) #+ list(self.coattention.parameters())# + list(self.conv_transfer.parameters()) #
        opt = torch.optim.Adam(total_params, lr=5e-4)
        scheduler = {'scheduler': torch.optim.lr_scheduler.StepLR(optimizer=opt, step_size=200, gamma=0.98),
                     'name': 'Learning Rate'}
        return [opt], [scheduler]





In [12]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

# from vio_attention_loader import PETRegNet_dataloader,PETRegNet_dataloader_dual
model = PETRegNet_dataloader().to(device)


from torchinfo import summary



MODEL_PATH = os.path.join('.','saved_model_100hrrt_att_32_timeembeding(test)')
print('MODEL_PATH={}'.format(MODEL_PATH))

# Initialise the LightningModule
# model = PETRegNet_dataloader(dropout=0.3)
# model(dropout=0.3)
# Set up loggers and checkpoints
tb_logger = pl.loggers.TensorBoardLogger(
    save_dir=os.path.join(MODEL_PATH,'logs')
)

lr_monitor = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=os.path.join(MODEL_PATH),
    filename="PETRegNet-{epoch}-{val_loss:.3f}",
    monitor='val_loss',
    save_last=True,
    save_top_k=2,
    mode='min'
)

# Check for last checkpoint
resume_checkpoint = None
if os.path.exists(os.path.join(MODEL_PATH,'last.ckpt')):
    resume_checkpoint = os.path.join(MODEL_PATH,'last.ckpt')

# Initialise Lightning's trainer.
trainer = pl.Trainer(
    accelerator='gpu',
    devices=[1],
    max_epochs=3000, #10k is generally enough for single-subjct studies
    logger=tb_logger,
    callbacks=[lr_monitor,checkpoint_callback],
    num_sanity_val_steps=1,
    check_val_every_n_epoch=20,
#     resume_from_checkpoint=resume_checkpoint # deprecated since v1.5
)

# Train
trainer.fit(model, train_loader, val_loader, ckpt_path=resume_checkpoint) # previously was trainer.fit(model, train_dataloader=tr_loader, val_dataloaders=val_loader)

MODEL_PATH=./saved_model_100hrrt_att_32_timeembeding(test)
2024-02-20 13:34:39,622 - GPU available: True (cuda), used: True
2024-02-20 13:34:39,623 - TPU available: False, using: 0 TPU cores
2024-02-20 13:34:39,624 - IPU available: False, using: 0 IPUs
2024-02-20 13:34:39,626 - HPU available: False, using: 0 HPUs
2024-02-20 13:34:39,630 - Missing logger folder: ./saved_model_100hrrt_att_32_timeembeding(test)/logs/lightning_logs
2024-02-20 13:34:39,634 - LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]
2024-02-20 13:34:39,645 - 
  | Name              | Type            | Params
------------------------------------------------------
0 | feature_extractor | UNet            | 859 K 
1 | coattention       | Cross_attention | 1.2 M 
2 | regression_layers | Sequential      | 133 K 
3 | loss_function     | MSELoss         | 0     
------------------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.838     Total estimated model p

Sanity Checking: 0it [00:00, ?it/s]

The number of training batches (9) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: 0it [00:00, ?it/s]

Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
### Testing on testing patients (for multisubject studies)

In [2]:
from evaluation_toolbox import build_df_results, show_df_loss, build_df_results_12, plot_vicra_network, plot_diff_vicra_network, save_synthetic_vicra, print_loss,plot_networks_comparison


In [3]:
test_set=['PF605', 'JB538', 'AF120', 'JG369', 'SY636',
'EC950', 'NM937', 'AS469', 'CH568', 'JO308',
'CJ509', 'SY869', 'BB688', 'HR322', 'DS636',
 'JR684', 'JM100', 'SM968', 'TM628', 'MC181']

In [4]:
from dataset_summary_toolbox import compute_delta_T 
from data_prep_toolbox import delta_T_magnitude, Relative_motion_A_to_B_12, build_legal_dataset, deal_dataframe, clean_df
from sampling_toolbox import data_split_sample, add_T_deltaT

In [5]:
summaries_test, delta_T_all_test=compute_delta_T(['FDG'], test_set,data_type='real')
df_test=[]
predictions=[]
df_input_diff_all=[]

for i in range(len(test_set)):
    df_test.append(deal_dataframe(summaries_test[i]))

In [6]:
KEYS = ['ThreeD_Cloud_ref','ThreeD_Cloud_mov'] 

train_transforms = Compose([
    LoadImaged(keys=KEYS, reader='NibabelReader', as_closest_canonical=False),
    AddChanneld(keys=KEYS), 
    Orientationd(keys=KEYS, axcodes='RAS'),
    ToTensord(keys=KEYS)    
])

<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.


In [8]:
class PETRegNet_dataloader(pl.LightningModule):
    
    def __init__(self, dropout=0.3):
        super().__init__()
        
        self.dropout = dropout
        
        self.feature_extractor = monai.networks.nets.resnet10(
            spatial_dims=3, 
            n_input_channels=1, 
            num_classes=128
        )

        # self.fwt_layers = torch.nn.Sequential(
        #     torch.nn.Linear(2, 32),
        #     torch.nn.LeakyReLU(),
        #     torch.nn.Dropout(dropout),
        #     torch.nn.Linear(32, 64),
        #     torch.nn.LeakyReLU(),
        #     torch.nn.Dropout(dropout),
        #     torch.nn.Linear(64, 128*2),            
        # )

        self.regression_layers = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            # torch.nn.LeakyReLU(),
            # torch.nn.Dropout(dropout),
            torch.nn.Linear(128, 16),
            # torch.nn.LeakyReLU(),
            # torch.nn.Dropout(dropout),
            torch.nn.Linear(16, 6),
        )
        
        self.loss_function = torch.nn.MSELoss()
        self.l1 = torch.nn.L1Loss(size_average=None, reduce=None, reduction = 'mean')
        self.sup_loss = []
        self.cod_loss = []
        self.epoch_num = 0
        
    def forward(self, x1, x2, x_t):
        y1 = self.feature_extractor(x1)
        y2 = self.feature_extractor(x2)
        y = torch.cat([y1,y2], dim=-1)

        # x_t=x_t.float()
        # x_fwt = self.fwt_layers(torch.transpose(x_t[None],0,1))
        # x_fwt=torch.squeeze(x_fwt)
        # y = torch.mul(y, x_fwt)
        y = self.regression_layers(y)
        return y

In [17]:
from network import PETRegNet_dataloader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
saved_model_path = '/data16/private/zc348/project/DL_HMC_attention/saved_model_100hrrt_dlhmc/PETRegNet-epoch=11909-val_loss=0.567.ckpt'
loaded_model = PETRegNet_dataloader().load_from_checkpoint(saved_model_path)
loaded_model.eval()
loaded_model.to(device)

PETRegNet_dataloader(
  (feature_extractor): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (layer2): Sequential(
      (0): ResNetBlock(
        (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
saved_model_path = '/data16/private/zc348/project/DL_HMC_attention/saved_model_100hrrt_att_32_stepsize200_resnetonly/PETRegNet-epoch=3689-val_loss=0.391.ckpt'
loaded_model = PETRegNet_dataloader().load_from_checkpoint(saved_model_path)
loaded_model.eval()
loaded_model.to(device)

PETRegNet_dataloader(
  (feature_extractor): ResNet(
    (conv1): Conv3d(1, 64, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): ResNetBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (layer2): Sequential(
      (0): ResNetBlock(
        (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
saved_model_path = '/data16/private/zc348/project/DL_HMC_attention/saved_model_100hrrt_att_32_subnumuber360_stepsize200/save/PETRegNet-epoch=1619-val_loss=0.426.ckpt'
loaded_model = PETRegNet_dataloader().load_from_checkpoint(saved_model_path)
loaded_model.eval()
loaded_model.to(device)


PETRegNet_dataloader(
  (feature_extractor): UNet(
    (conv_encode1): Sequential(
      (0): Conv3d(1, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (conv_maxpool1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv_encode2): Sequential(
      (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    

In [18]:
from copy import deepcopy
df_results_all=[]
y_list_all=[]
df_input_diff_all=[]
prediction_list = list()
times = torch.zeros((1800*20,1)) 
# torch.cuda.synchronize()
# starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
idx=0

for j in range(len(test_set)):

    df = df_test[j]

    #fixed reference time
    df_input_diff = vicra_toolbox.build_netinput_fixed_reference(df).reset_index()
    pairs=[]
    for i in range(len(df_input_diff)):
        pairs.append(np.array([df_input_diff['ScanStart_ref'][i], df_input_diff['ScanStart_mov'][i]]))
    df_input_diff['pairs']=pairs
    
    df_input_diff_all.append(df_input_diff)

    ##building testing dataloader
    test_dict = df_input_diff.to_dict('records')

    for i in range(len(test_dict)):
        x = test_dict[i]['ThreeD_Cloud_ref'].find('nii')
        fn_cloud1 = test_dict[i]['ThreeD_Cloud_ref'][0:x] + 'nii_monai_resize'
        x = x+3
        y = test_dict[i]['ThreeD_Cloud_ref'].find('3dcld')
        fn_cloud2 =  test_dict[i]['ThreeD_Cloud_ref'][x:y] + '3dcld_monai_rz.nii'
        test_dict[i]['ThreeD_Cloud_ref'] = fn_cloud1 + fn_cloud2

        x = test_dict[i]['ThreeD_Cloud_mov'].find('nii')
        fn_cloud1 = test_dict[i]['ThreeD_Cloud_mov'][0:x] + 'nii_monai_resize'
        x = x+3
        y = test_dict[i]['ThreeD_Cloud_mov'].find('3dcld')
        fn_cloud2 =  test_dict[i]['ThreeD_Cloud_mov'][x:y] + '3dcld_monai_rz.nii'
        test_dict[i]['ThreeD_Cloud_mov'] = fn_cloud1 + fn_cloud2

    # Create the Dataset
#     ds_test = monai.data.CacheDataset(data=test_dict, transform=train_transforms)
    ds_test = monai.data.Dataset(data=test_dict, transform=train_transforms)
    #ds_tr = monai.data.SmartCacheDataset(data=tr_dict,transform=train_transforms,replace_rate=1,cache_num=64,shuffle=True)
    # Create the DataLoader
    test_loader = monai.data.DataLoader(ds_test, batch_size=32, num_workers=2, collate_fn=list_data_collate)
    
    #calculate loss function and network output
    # saved_model_path = os.path.join(MODEL_PATH,'PETRegNet-epoch=3130-val_loss=0.454.ckpt')
    # loaded_model = PETRegNet.load_from_checkpoint(saved_model_path)


    loss = 0
    loss_list = list()
    time_list = list()
    
    y_list = list()
    loss1 = []
    

    for test_data in test_loader:
        x1 = test_data['ThreeD_Cloud_ref'].to(device)
        x2 = test_data['ThreeD_Cloud_mov'].to(device)
        x_t = test_data['pairs'].to(device)
        time = test_data['delta_t']
        y = test_data['T'].cpu().numpy()
        # starter.record()
        y_test = loaded_model(x1, x2,x_t).detach().cpu().numpy()
        torch.cuda.synchronize() 
        loss = y-y_test
        l = len(loss)
        for j in range(l):
            loss1=sum(np.square(loss[j]))/len(loss[j])
            loss_list.append(loss1)
            time_list.append(time.numpy()[j])
            prediction_list.append(y_test[j])
            y_list.append(y[j])
        del test_data
        idx= idx+1
        
    
    df_results = pd.DataFrame()
    y_list_all.append(y_list)
    
    df_results['Time'] = time_list
    df_results['Loss'] = loss_list
    df_results_all.append(df_results)

In [19]:
df_results_test = build_df_results(test_set, df_input_diff_all, prediction_list).reset_index(drop=True)

In [13]:
def iqr(data):
    # data = list(data)
    q1 = np.percentile(data, 25, interpolation='midpoint')
    q3 = np.percentile(data, 75, interpolation='midpoint')

    iqr = q3 - q1
    return iqr

In [20]:
tp = np.stack(df_results_test['model parameters'])
tv = np.stack(df_results_test['vicra parameters'])
# tp = np.stack(total_model)
# tv = np.stack(total_vicra)
tx = np.square(tv[:,0]-tp[:,0])
ty = np.square(tv[:,1]-tp[:,1])
tz = np.square(tv[:,2]-tp[:,2])

rx = np.square(tv[:,3]-tp[:,3])
ry = np.square(tv[:,4]-tp[:,4])
rz = np.square(tv[:,5]-tp[:,5])

mtx = np.mean(tx) 
mty = np.mean(ty) 
mtz = np.mean(tz)
mt = (mtx + mty +mtz)/3  

mrx = np.mean(rx) 
mry = np.mean(ry) 
mrz = np.mean(rz)
mr = (mrx + mry + mrz)/3 

st_list = []
sr_list = []
mmt_list = []
mmr_list = []
met_list=[]
mer_list = []
iqt_list=[]
iqr_list=[]
tmt_list=[]
tst_list=[]
tmet_list=[]
tit_list=[]
for i in range(20):
    mmtx = np.mean(tx[i*1800:(i+1)*1800]) 
    mmty = np.mean(ty[i*1800:(i+1)*1800]) 
    mmtz = np.mean(tz[i*1800:(i+1)*1800])
    mmt = (mmtx + mmty +mmtz)/3  
    mmt_list.append(mmt)

    mmrx = np.mean(rx[i*1800:(i+1)*1800]) 
    mmry = np.mean(ry[i*1800:(i+1)*1800]) 
    mmrz = np.mean(rz[i*1800:(i+1)*1800])
    mmr = (mmrx + mmry + mmrz)/3 
    mmr_list.append(mmr)


    stx = np.std(tx[i*1800:(i+1)*1800]) 
    sty = np.std(ty[i*1800:(i+1)*1800]) 
    stz = np.std(tz[i*1800:(i+1)*1800])
    st = (stx + sty +stz)/3  
    st_list.append(st)

    srx = np.std(rx[i*1800:(i+1)*1800]) 
    sry = np.std(ry[i*1800:(i+1)*1800]) 
    srz = np.std(rz[i*1800:(i+1)*1800])
    sr = (srx + sry + srz)/3 
    sr_list.append(sr)

    metx = np.median(tx[i*1800:(i+1)*1800]) 
    mety = np.median(ty[i*1800:(i+1)*1800]) 
    metz = np.median(tz[i*1800:(i+1)*1800])
    met = (metx + mety +metz)/3  
    met_list.append(met)

    merx = np.median(rx[i*1800:(i+1)*1800]) 
    mery = np.median(ry[i*1800:(i+1)*1800]) 
    merz = np.median(rz[i*1800:(i+1)*1800])
    mer = (merx + mery +merz)/3  
    mer_list.append(mer)

    iqtx = iqr(tx[i*1800:(i+1)*1800]) 
    iqty = iqr(ty[i*1800:(i+1)*1800]) 
    iqtz = iqr(tz[i*1800:(i+1)*1800])
    iqt = (iqtx + iqty +iqtz)/3  
    iqt_list.append(iqt)

    iqrx = iqr(rx[i*1800:(i+1)*1800]) 
    iqry = iqr(ry[i*1800:(i+1)*1800]) 
    iqrz = iqr(rz[i*1800:(i+1)*1800])
    iqrr = (iqrx + iqry +iqrz)/3  
    iqr_list.append(iqrr)
    

    tmx = np.mean(tx[i*1800:(i+1)*1800]+rx[i*1800:(i+1)*1800]) 
    tmy = np.mean(ty[i*1800:(i+1)*1800]+ry[i*1800:(i+1)*1800]) 
    tmz = np.mean(tz[i*1800:(i+1)*1800]+rz[i*1800:(i+1)*1800])
    tmt = (tmx + tmy + tmz)/3  
    tmt_list.append(tmt)
    
    tsx = np.std(tx[i*1800:(i+1)*1800]+rx[i*1800:(i+1)*1800]) 
    tsy = np.std(ty[i*1800:(i+1)*1800]+ry[i*1800:(i+1)*1800]) 
    tsz = np.std(tz[i*1800:(i+1)*1800]+rz[i*1800:(i+1)*1800])
    tst = (tsx + tsy + tsz)/3  
    tst_list.append(tst)

    tmex = np.median(tx[i*1800:(i+1)*1800]+rx[i*1800:(i+1)*1800]) 
    tmey = np.median(ty[i*1800:(i+1)*1800]+ry[i*1800:(i+1)*1800]) 
    tmez = np.median(tz[i*1800:(i+1)*1800]+rz[i*1800:(i+1)*1800])
    tmet = (tmex + tmey + tmez)/3  
    tmet_list.append(tmet)
    
    tix = iqr(tx[i*1800:(i+1)*1800]+rx[i*1800:(i+1)*1800]) 
    tiy = iqr(ty[i*1800:(i+1)*1800]+ry[i*1800:(i+1)*1800]) 
    tiz = iqr(tz[i*1800:(i+1)*1800]+rz[i*1800:(i+1)*1800])
    tit = (tix + tiy + tiz)/3  
    tit_list.append(tit)


the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)
the `interpolation=` argument to percentile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they used. (Deprecated NumPy 1.22)


In [15]:
def cal(st_list, sr_list, mmt_list, mmr_list, met_list, mer_list, iqt_list, iqr_list, tmt_list, tst_list, tmet_list, tit_list):
   mt,mr,st,sr,met,mer,it,ir,tm,ts,tme,tq =np.mean(mmt_list),np.mean(mmr_list),np.mean(st_list),np.mean(sr_list),np.mean(met_list), np.mean(mer_list),np.mean(iqt_list), np.mean(iqr_list),np.mean(tmt_list), np.mean(tst_list),np.mean(tmet_list), np.mean(tit_list)
   return print(mt,st,met,it,mr,sr,mer,ir,tm,ts,tme,tq)

In [24]:
cal(st_list, sr_list, mmt_list, mmr_list, met_list, mer_list, iqt_list, iqr_list, tmt_list, tst_list, tmet_list, tit_list)

2.31551955807927 5.257200731721682 1.3593691154761676 2.6634826902675397 2.8423442085235258 3.3530695008560345 2.4823552508772457 4.140984513256411 5.157863766602796 7.759214748962546 4.180926186025017 6.569022051085443


In [16]:
cal(st_list, sr_list, mmt_list, mmr_list, met_list, mer_list, iqt_list, iqr_list, tmt_list, tst_list, tmet_list, tit_list)

8.734676492050678 12.438232688024492 5.455695533007011 10.373605568641922 4.785129250333939 5.275023775152056 3.7974137972547863 7.438375331102343 13.519805742384616 16.634821037950694 9.774156189144232 17.46659115807403


In [21]:
cal(st_list, sr_list, mmt_list, mmr_list, met_list, mer_list, iqt_list, iqr_list, tmt_list, tst_list, tmet_list, tit_list)

8.644660552343058 12.176103419218897 5.523794180872654 10.281121586120358 4.858054341860089 5.325544347075972 3.873080803321316 7.617762165329213 13.502714894203146 16.42654734896903 9.920247395572273 17.43873954391016


In [16]:
for i in test_set:
    save_synthetic_vicra('FDG', i, df_results_test, [3600,5399], 'test')